<a href="https://colab.research.google.com/github/Mike030668/MIPT_magistratura/blob/main/NLP/DZ_3_NER_bert_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token Classification. Практическое задание (PJ)

Для закрепления материала модуля вам необходимо решить задачу NER для предоставленного датасета, используя любые доступные вам средства. Модель должна обучаться на файле `train.txt`, валидироваться на файле `dev.txt`, а её качество необходимо оценить на файле `test.txt`.

Для достижения наилучшего результата уделите внимание подбору гиперпарметров как в плане архитектуры, так и в плане обучения модели.

Критерии оценивания проекта:

- общее качество кода и следование PEP-8;
- использование рекуррентных сетей;
- использованы варианты архитектур, близкие к state of the art для данной задачи;
- произведен подбор гиперпараметров;
- использованы техники изменения learning rate (lr scheduler);
- использована адекватная задаче функция потерь;
- использованы техники регуляризации;
- корректно проведена валидация модели;
- использованы техники ensemble;
- использованы дополнительные данные;

- итоговое значение метрики качества > 0.6 (f1)

[датасет](https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip)

In [1]:
import numpy as np
import pandas as pd

In [2]:
loaded_path = '/content/data.zip'
!wget https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip -O {loaded_path}


--2023-10-04 14:14:40--  https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip
Resolving lms.skillfactory.ru (lms.skillfactory.ru)... 51.250.7.2
Connecting to lms.skillfactory.ru (lms.skillfactory.ru)|51.250.7.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870425 (850K) [application/x-zip-compressed]
Saving to: ‘/content/data.zip’

/content/data.zip   100%[===================>] 850.02K  1007KB/s    in 0.8s    

2023-10-04 14:14:42 (1007 KB/s) - ‘/content/data.zip’ saved [870425/870425]



In [3]:
!unzip {loaded_path}

Archive:  /content/data.zip
  inflating: dev.txt                 
  inflating: test.txt                
  inflating: train.txt               


In [4]:
import json

def txt2json(file_path):

    save_path = f"{file_path.split('.')[0]}"+'.json'
    out_l = []
    with open(file_path, 'r') as in_file:

      stripped = (line.strip('\n') for line in in_file)
      line_dict = dict()

      txt = []
      lbl = []
      j = 0
      for i, line in enumerate(stripped):
        if line:
          t, l =  line.split(' ')
          txt.append(t)
          lbl.append(l)
        if not line:
          line_dict['token'] = txt
          line_dict['ner'] = lbl
          out_l.append(line_dict)
          txt = []
          lbl = []
          line_dict = dict()
          j+=1

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(out_l, f, ensure_ascii=False, indent=4)





In [5]:
txt2json('train.txt')
txt2json('dev.txt')
txt2json('test.txt')

In [6]:
!pip install datasets -q
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.5 MB/s eta 0:00:00


In [7]:
datasets = load_dataset("json",
                       data_files={"train": "train.json", 'valid': "dev.json", "test": "test.json"},
                       )

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
datasets['train'], datasets['valid'], datasets['test']

(Dataset({
     features: ['token', 'ner'],
     num_rows: 7746
 }),
 Dataset({
     features: ['token', 'ner'],
     num_rows: 2582
 }),
 Dataset({
     features: ['token', 'ner'],
     num_rows: 2582
 }))

In [9]:
datasets['train'][2]

{'token': ['В',
  'Ханты-Мансийском',
  'автономном',
  'округе',
  'с',
  'должности',
  'снят',
  'начальник',
  'УВД',
  'Николай',
  'Гудожников',
  '.'],
 'ner': ['O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'B-PER',
  'I-PER',
  'O']}

In [10]:
from datasets import ClassLabel, Sequence
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [11]:
show_random_elements(datasets["train"])

,token,ner
0,"[Пресс-секретарь, президента, РФ, Дмитрий, Песков, опроверг, утверждения, о, том, ,, что, Владимир, Путин, якобы, встречался, в, Петербурге, во, время, международного, экономического, форума, с, экс-главой, Минобороны, Анатолием, Сердюковым, .]","[O, O, B-LOC, B-PER, I-PER, O, O, O, O, O, O, B-PER, I-PER, O, O, O, B-LOC, O, O, O, O, O, O, O, B-ORG, B-PER, I-PER, O]"
1,"[Решение, покинуть, сенат, было, согласовано, с, президентом, Чечни, Рамзаном, Кадыровым, еще, в, июле, ,, признались, в, республиканском, правительстве, .]","[O, O, O, O, O, O, O, B-LOC, B-PER, I-PER, O, O, O, O, O, O, O, O, O]"
2,"[Как, сообщили, ИА, REGNUM, в, пресс-службе, Генеральной, прокуратуры, 28, июня, ,, соответствующий, приказ, подписал, в, минувшую, среду, исполняющий, обязанности, Генерального, прокурора, Российской, Федерации, Александр, Буксман, .]","[O, O, B-ORG, I-ORG, O, O, B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, I-LOC, B-PER, I-PER, O]"
3,"[Дополнение, :, Д.Медведев, присвоил, звания, сотрудников, полиции, и, переназначил, 14, руководителей, УВД, ,, ГУВД, и, МВД, по, субъектам, РФ, .]","[O, O, B-PER, O, O, O, O, O, O, O, O, B-ORG, O, B-ORG, O, B-ORG, O, O, B-LOC, O]"
4,"[М.Бади, –, ветеринар, по, образованию, –, еще, в, 1960-е, годы, активно, сотрудничал, с, радикалами, ,, выступавшими, против, мирных, переговоров, с, Израилем, .]","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O]"
5,"[подписал, премьер-министр, РФ, Владимир, Путин, .]","[O, O, B-LOC, B-PER, I-PER, O]"
6,"[Перед, тем, Дэйли, занимал, пост, министра, торговли, страны, и, работал, топ-менеджером, банка, JP, Morgan, .]","[O, O, B-PER, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O]"
7,"[Сообщалось, ,, что, после, своей, отставки, Гайтнер, может, стать, главой, Федерального, резервного, банка, Нью-Йорка, .]","[O, O, O, O, O, O, B-PER, O, O, O, B-ORG, I-ORG, I-ORG, B-LOC, O]"
8,"[Ранее, он, заявлял, ,, что, желает, покинуть, пост, советника, президента, США, в, течение, двух, лет, .]","[O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O]"
9,"[Кандидатура, Немцовой, оказалась, приемлема, и, для, LIDEM, ,, и, для, TOP, 09, .]","[O, B-PER, O, O, O, O, B-ORG, O, O, O, B-ORG, I-ORG, O]"


In [12]:
def flat_lists(seqs):
  out = []
  for s in seqs:
    out.extend(s)
  return out

In [13]:
NER_labels = list(set(flat_lists(datasets["train"]['ner'])))
NUM_CLASSES = len(NER_labels)
print(f'NUM_CLASSES = {NUM_CLASSES}')

NUM_CLASSES = 7


In [14]:
dict_lbl_to_ner = {k: v for k, v in enumerate(NER_labels)}
dict_ner_to_lbl = {v: k for k, v in dict_lbl_to_ner.items()}
print(f'NER to lbl dict - {dict_ner_to_lbl}')

NER to lbl dict - {'O': 0, 'I-PER': 1, 'B-ORG': 2, 'I-ORG': 3, 'B-PER': 4, 'B-LOC': 5, 'I-LOC': 6}


# RU_bert
- https://colab.research.google.com/github/shitkov/bert4classification/blob/main/bert4clssification.ipynb#scrollTo=KoB7lDv_8AWe

- https://github.com/shitkov/bert4classification/blob/main/bert_classifier.py

- https://habr.com/ru/articles/567028/


In [15]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00


In [16]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

In [17]:
class CustomDataset(Dataset):

  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = self.targets[idx]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [27]:
class BertClassifier:

    def __init__(self, model_path, tokenizer_path, n_classes=2, epochs=1, model_save_path='/content/bert.pt'):
        self.model = BertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.out_features = self.model.bert.encoder.layer[1].output.dense.out_features
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)

    def preparation(self, X_train, y_train, X_valid, y_valid):
        # create datasets
        self.train_set = CustomDataset(X_train, y_train, self.tokenizer)
        self.valid_set = CustomDataset(X_valid, y_valid, self.tokenizer)

        # create data loaders
        self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=2, shuffle=True)

        # helpers initialization
        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(self.train_loader) * self.epochs
            )
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)

    def fit(self):
        self.model = self.model.train()
        losses = []
        correct_predictions, bad_predictions = 0, 0

        for data in self.train_loader:
            input_ids = data["input_ids"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)
            bad_predictions += torch.sum(preds != targets)

            losses.append(loss.item())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        #train_acc = correct_predictions.double() / len(self.train_set)
        train_acc = correct_predictions.double() / (correct_predictions.double() + bad_predictions.double())
        train_loss = np.mean(losses)
        return train_acc, train_loss

    def eval(self):
        self.model = self.model.eval()
        losses = []
        correct_predictions, bad_predictions = 0, 0

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                bad_predictions += torch.sum(preds != targets)
                losses.append(loss.item())

        val_acc = correct_predictions.double() / (correct_predictions.double() + bad_predictions.double())
        #val_acc = correct_predictions.double() / len(self.valid_set)
        val_loss = np.mean(losses)
        return val_acc, val_loss

    def train(self):
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss} accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss} accuracy {val_acc}')
            print('-' * 10)

            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path)

    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }

        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)

        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )

        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

# Initialize BERT classifier

In [28]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=NUM_CLASSES,
        epochs=2,
        model_save_path='/content/bert.pt'
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Data for model

In [29]:
X_train = flat_lists(datasets['train']['token'])
Y_train = [dict_ner_to_lbl[ner] for ner in flat_lists(datasets['train']['ner'])]

X_valid = flat_lists(datasets['valid']['token'])
Y_valid = [dict_ner_to_lbl[ner] for ner in flat_lists(datasets['valid']['ner'])]

classifier.preparation(
        X_train=X_train,
        y_train=Y_train,
        X_valid=X_valid,
        y_valid=Y_valid)


# Train loop

In [30]:
classifier.train()

Epoch 1/2
Train loss 0.2719552842008537 accuracy 0.9379092475588742
Val loss 0.2516265884513306 accuracy 0.9457468839135701
----------
Epoch 2/2
Train loss 0.21628010678030626 accuracy 0.9520582040972622
Val loss 0.24988452966985128 accuracy 0.9484233970226233
----------


# Check test data

In [31]:
texts = flat_lists(datasets['test']['token'])
labels = [dict_ner_to_lbl[ner] for ner in flat_lists(datasets['test']['ner'])]

predictions = [classifier.predict(t) for t in texts]

In [32]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

precision, recall, f1score = precision_recall_fscore_support(labels, predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.8030295362386545, recall: 0.746459171229349, f1score: 0.7581487587875906


In [33]:
from sklearn.metrics import classification_report

In [34]:
print(classification_report(labels, predictions, target_names=NER_labels))

              precision    recall  f1-score   support

           O       0.98      0.99      0.99     44847
       I-PER       0.64      0.74      0.69      1367
       B-ORG       0.82      0.86      0.84      1734
       I-ORG       0.72      0.26      0.38      1090
       B-PER       0.82      0.78      0.80      2150
       B-LOC       0.84      0.89      0.86      1508
       I-LOC       0.80      0.70      0.74       342

    accuracy                           0.95     53038
   macro avg       0.80      0.75      0.76     53038
weighted avg       0.95      0.95      0.95     53038

